<a href="https://colab.research.google.com/github/Antonio14-code/Seguridad.facial/blob/main/Seguridad_por_rostro.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!pip install google-generativeai opencv-python

In [ ]:
!pip install fpdf

  Preparing metadata (setup.py) ... done
  Created wheel for fpdf: filename=fpdf-1.7.2-py2.py3-none-any.whl size=40704 sha256=35e8f6359d84fe2e3df7bff0a7e361cf51269e4938112112f23abf3c7771e16b
  Stored in directory: /root/.cache/pip/wheels/65/4f/66/bbda9866da446a72e206d6484cd97381cbc7859a7068541c36
Successfully built fpdf


In [ ]:
import cv2
import google.generativeai as genai
from google.colab import files

In [ ]:
API_KEY = "TU API KEY AQUI"
genai.configure(api_key=API_KEY)

In [ ]:
def subir_imagen():
    print("📂 Sube una imagen para analizar:")
    archivo = files.upload()
    return list(archivo.keys())[0]

🔹 Función para detectar rostros con OpenCV

In [ ]:
def detectar_rostro_opencv(imagen_path):
    imagen = cv2.imread(imagen_path)

    if imagen is None:
        print("❌ Error: No se pudo cargar la imagen.")
        return []

    # Convertir la imagen a escala de grises
    gris = cv2.cvtColor(imagen, cv2.COLOR_BGR2GRAY)

    # Cargar el detector de rostros de OpenCV
    detector = cv2.CascadeClassifier(cv2.data.haarcascades + "haarcascade_frontalface_default.xml")

    # Detectar rostros en la imagen
    rostros = detector.detectMultiScale(gris, scaleFactor=1.1, minNeighbors=5, minSize=(30, 30))

    return rostros

🔹 Función para generar políticas de seguridad con Gemini

In [ ]:
from fpdf import FPDF

def generar_politicas_seguridad():
    prompt = (
        "Genera un documento de políticas de seguridad para EMPRESARIOTEC S.A. "
        "basado en los siguientes puntos clave:\n"
        "- Implementación de control de acceso biométrico con reconocimiento facial.\n"
        "- Monitoreo en tiempo real mediante cámaras de seguridad.\n"
        "- Definición de protocolos claros para la gestión de accesos y respuestas a incidentes.\n"
        "- Protección de activos físicos y digitales para garantizar la continuidad operativa.\n"
    )
    try:
        model = genai.GenerativeModel("gemini-1.5-pro")
        respuesta = model.generate_content(prompt)
        politicas = respuesta.text

        # Guardar el resultado en un archivo PDF
        pdf = FPDF()
        pdf.set_auto_page_break(auto=True, margin=15)
        pdf.add_page()
        pdf.set_font("Arial", style='B', size=16)
        pdf.cell(200, 10, "Políticas de Seguridad - EMPRESARIOTEC S.A.", ln=True, align='C')

        pdf.set_font("Arial", size=12)
        pdf.ln(10)  # Salto de línea
        for linea in politicas.split("\n"):
            pdf.multi_cell(0, 10, linea)  # Agregar texto con salto de línea automático
            pdf.ln(2)

        pdf_filename = "Politicas_Seguridad.pdf"
        pdf.output(pdf_filename)

        print(f"✅ Políticas de seguridad generadas y guardadas en: {pdf_filename}")
        return pdf_filename

    except Exception as e:
        print(f"❌ Error al generar políticas de seguridad: {e}")
        return None

🔹 Ejecución del programa

In [ ]:
from langchain.tools import BaseTool
from typing import Type
from pydantic import BaseModel, Field
import openai # Asegúrate que esté configurado con tu API Key

class DallEImageGeneratorInput(BaseModel):
    prompt: str = Field(description="El prompt para generar la imagen")

class DallEImageGeneratorTool(BaseTool):
    name = "DallEImageGenerator"
    description = "Útil para generar imágenes a partir de un prompt de texto usando DALL-E."
    args_schema: Type[BaseModel] = DallEImageGeneratorInput

    def _run(self, prompt: str) -> str:
        try:
            client = openai.OpenAI() # Asume que la API key ya está configurada en el entorno
            response = client.images.generate(
                model="dall-e-3",
                prompt=prompt,
                n=1,
                size="1024x1024",
                response_format="url"
            )
            return response.data[0].url
        except Exception as e:
            return f"Error al generar imagen: {str(e)}"

    async def _arun(self, prompt: str) -> str:
        # Implementación asíncrona si es necesario
        raise NotImplementedError("DallEImageGeneratorTool no soporta ejecución asíncrona todavía")

# Uso del Tool (por ejemplo, en un agente)
# image_tool = DallEImageGeneratorTool()
# image_url = image_tool.run(prompt="un perro futurista en Marte")
# print(image_url)

📂 Sube una imagen para analizar:


Saving 4.jpg to 4.jpg
🔍 Detectando rostros en la imagen subida...
✅ Rostros detectados: 1
📜 Generando políticas de seguridad...
✅ Políticas de seguridad generadas y guardadas en: Politicas_Seguridad.pdf
Politicas_Seguridad.pdf
